# Intro_to_SQLite

Very often, the best way to store spreadsheet-like data is in a CSV. Sometimes it's useful to store data in a database. The main reason to do this would be so that you can make complex or pre-existing SQL queries against the data, or join datasets from different places.

Note, however, that many of these things can be done in `pandas` too, and you may prefer learning `pandas` to learning SQL.

In [77]:
tops = {
    'Abenaki Fm': 3404.31,
    'Base O-Marker': 2469.20,
    'Dawson Canyon Fm': 984.50,
    'Logan Canyon Fm': 1136.90,
    'Lower Baccaro': 3964.53,
    'Lower Missisauga Fm': 3190.64,
    'Mid Baccaro': 3485.08,
    'Upper Missisauga Fm': 2251.25,
    'Wyandot Fm': 867.15
}

We need a design for the database. Let's try this:

- **well** id, name, location, 
- **formations**  id, name, begin, end
- **tops**  id, formation_id, well_id, depth
- **periods**  id, name, begin, end

First we create the database, which in SQLite is stored in a flat file:

In [78]:
import sqlite3 as lite
import uuid

con = lite.connect('../data/strat.db')

In [79]:
sql = """CREATE TABLE wells (
               id TEXT PRIMARY KEY,
               uwi TEXT,
               lat DECIMAL,
               lon DECIMAL
         )"""

In [80]:
with con:
    cur = con.cursor()    
    cur.execute(sql)
    
    uid = str(uuid.uuid4())
    uwi = "L-30"
    lat = lon = 0.0
    cur.execute("INSERT INTO wells VALUES('{}','{}',{},{})".format(uid, uwi, lat, lon))

In [81]:
with con:    
    cur = con.cursor()    
    cur.execute("SELECT id FROM wells WHERE uwi='L-30'")
    r = cur.fetchall()
r

[('76a31ded-c1c2-4e96-8f92-94412de275b3',)]

In [82]:
sql = """CREATE TABLE formations (
               id TEXT PRIMARY KEY,
               name TEXT,
               begin DECIMAL,
               end DECIMAL
         )"""

In [83]:
with con:
    cur = con.cursor()    
    cur.execute(sql)
    
    for name in tops.keys():
        uid = str(uuid.uuid4())
        name = name.replace(' Fm', '')
        begin = end = 0.0
        cur.execute("INSERT INTO formations VALUES('{}','{}',{},{})".format(uid, name, begin, end))

In [84]:
sql = """CREATE TABLE tops (
               id TEXT PRIMARY KEY,
               well_id TEXT,
               formation_id TEXT,
               depth DECIMAL
         )"""

In [85]:
with con:
    cur = con.cursor()    
    cur.execute(sql)
    
    for name, depth in tops.items():
        uid = str(uuid.uuid4())
        well_id = r[0][0]
        try:
            name = name.replace(' Fm', '')
            cur.execute("SELECT id FROM formations WHERE name LIKE '{}%'".format(name))
            formation_id = cur.fetchall()[0][0]
        except ValueError:
            print('Skipped {}, not in database'.format(name))
            continue
        cur.execute("INSERT INTO tops VALUES('{}','{}','{}',{})".format(uid, well_id, formation_id, depth))

Now we can query it. Let's write the SQL first.

In [106]:
sql = """SELECT depth 
         FROM   tops 
         WHERE  formation_id IN (SELECT id 
                                 FROM   formations 
                                 WHERE  name LIKE '%{}%') """.format('Baccaro')

In [107]:
with con:    
    cur = con.cursor()    
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)

(3964.53,)
(3485.08,)
